# Adapters

> Adapter implementations for integrating with plugin registries

In [ ]:
#| default_exp core.adapters

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from typing import Dict, Any, List, Optional

from cjm_plugin_system.core.manager import PluginManager
from cjm_plugin_system.core.metadata import PluginMeta

from cjm_fasthtml_workflow_transcription_single_file.core.config import SingleFileWorkflowConfig
from cjm_fasthtml_workflow_transcription_single_file.core.protocols import PluginInfo, PluginRegistryProtocol

## PluginRegistryAdapter Class

Adapter that wraps a `PluginManager` (from `cjm-plugin-system`) to implement `PluginRegistryProtocol`. This allows the workflow to work with the new plugin manager architecture while maintaining compatibility with workflow components that expect the registry protocol.

In [ ]:
#| export
class PluginRegistryAdapter:
    """Adapts PluginManager to workflow's PluginRegistryProtocol."""

    def __init__(self,
                 plugin_manager: PluginManager,  # The PluginManager instance to wrap
                 config: SingleFileWorkflowConfig, # The workflow config instance
                 category: str = "transcription"  # Plugin category to filter by
                 ):
        """Initialize the adapter."""
        self._manager = plugin_manager
        self._config = config
        self._category = category

    def _meta_to_info(self, meta: PluginMeta) -> PluginInfo:
        """Convert PluginMeta to PluginInfo."""
        return PluginInfo(
            id=meta.name,
            name=meta.name,
            title=meta.config_schema.get('title', meta.name) if meta.config_schema else meta.name,
            is_configured=True,  # All discovered plugins are "configured" via manifest
            supports_streaming=self._check_streaming_support(meta)
        )

    def get_configured_plugins(self) -> List[PluginInfo]:  # List of PluginInfo for configured plugins
        """Get all configured transcription plugins (all discovered are considered configured)."""
        metas = self._manager.get_discovered_by_category(self._category)
        return [self._meta_to_info(meta) for meta in metas]

    def get_all_plugins(self) -> List[PluginInfo]:  # List of PluginInfo for all discovered plugins
        """Get all discovered transcription plugins."""
        metas = self._manager.get_discovered_by_category(self._category)
        return [self._meta_to_info(meta) for meta in metas]

    def get_plugin(self,
                   plugin_id: str  # Unique plugin identifier (name)
                   ) -> Optional[PluginInfo]:  # PluginInfo if found, None otherwise
        """Get a specific plugin by ID."""
        meta = self._manager.get_discovered_meta(plugin_id)
        if not meta or meta.category != self._category:
            return None

        return self._meta_to_info(meta)

    def get_plugin_config(self,
                          plugin_id: str  # Unique plugin identifier
                          ) -> Dict[str, Any]:  # Configuration dictionary
        """Get the configuration for a plugin."""

        config_file = self._config.plugin_config_dir / f"{plugin_id}.json"
        if config_file.exists():
            with open(config_file, 'r') as f:
                plugin_config = json.load(f)
                return plugin_config

        return self._manager.get_plugin_config(plugin_id) or {}

    def _check_streaming_support(self,
                                 meta: PluginMeta  # Plugin metadata object
                                 ) -> bool:  # True if the plugin supports streaming
        """Check if a plugin supports streaming output."""
        if meta.config_schema:
            properties = meta.config_schema.get("properties", {})
            return "stream_output" in properties or "streaming" in properties
        return False

## Removed: DefaultConfigPluginRegistryAdapter

The `DefaultConfigPluginRegistryAdapter` class is no longer needed because:
- Plugin configuration is now embedded in manifest files during `cjm-ctl install-all`
- Default values are extracted from the JSON schema in the manifest
- The `PluginManager` handles configuration management directly

In [ ]:
# DefaultConfigPluginRegistryAdapter has been removed.
# Configuration is now handled via manifest files and PluginManager.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()